# Модуль 25. A/B-тестирование (HW-03)

## ПОСТАНОВКА ЗАДАЧИ

К вам попали результаты A/A/B-тестирования от одного известного маркетплейса. 

>[sample_a](https://lms-cdn.skillfactory.ru/assets/courseware/v1/458c3852d20031834c4d62a99ea424ef/asset-v1:SkillFactory+DSPRML+ALWAYS+type@asset+block/sample_a.zip), [sample_c](https://lms-cdn.skillfactory.ru/assets/courseware/v1/a22e256e36aac4608cc65858614ccdbf/asset-v1:SkillFactory+DSPRML+ALWAYS+type@asset+block/sample_c.zip) — АА-группы, [sample_b](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b02dee68c33d49f27793522f89c50cab/asset-v1:SkillFactory+DSPRML+ALWAYS+type@asset+block/sample_b.zip) — отдельная группа. 

- [Скачать item_prices](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c2567f1976c44ff69cc18057ba824dd4/asset-v1:SkillFactory+DSPRML+ALWAYS+type@asset+block/item_prices.zip)

В каждом датасете есть три типа действий пользователей: 0 — клик, 1 — просмотр и 2 — покупка (пользователь просматривает выдачу товаров, кликает на понравившийся товар и совершает покупку).

Маркетплейс ориентируется на следующие метрики:

- ctr (отношение кликов к просмотрам товаров);
- purchase rate (отношение покупок к просмотрам товаров);
- gmv (оборот, сумма произведений количества покупок на стоимость покупки), где считаем 1 сессию за 1 точку (1 сессия на 1 пользователя).

Данные уже почищены по сессиям, вы можете использовать их в агрегированном виде. Ваша задача — понять, нет ли проблемы с разъезжанием сплитов и улучшает ли алгоритм B работу маркетплейса.

Тест Шапиро-Уилка проведите на $alpha = 0.01$.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df_a = pd.read_csv('sample_a.zip').copy()
print(df_a.shape)
df_a.sort_values(['user_id'])

(1188912, 3)


,user_id,item_id,action_id
833360,241,8589,1
726761,241,1679,1
105344,241,5545,1
977034,241,3006,1
741420,241,6424,2
...,...,...,...
446113,99880,8669,1
609097,99880,3078,1
662502,99880,729,1
6729,99880,6447,1


In [3]:
df_c = pd.read_csv('sample_c.zip').copy()
print(df_c.shape)
df_c.sort_values(['user_id'])

(1205510, 3)


,user_id,item_id,action_id
1035660,200225,6206,1
68224,200225,8428,1
558777,200225,6728,1
1029160,200225,190,1
516598,200225,3603,1
...,...,...,...
75883,299828,4196,1
1021986,299828,898,1
956766,299828,96,2
432462,299828,8944,0


In [4]:
df_b = pd.read_csv('sample_b.zip').copy()
print(df_b.shape)
df_b.sort_values(['user_id'])

(1198438, 3)


,user_id,item_id,action_id
582303,100038,687,0
14141,100038,5392,1
611528,100038,3456,1
474279,100038,6917,1
1161252,100038,5704,1
...,...,...,...
188671,199966,810,0
731750,199966,6180,1
879316,199966,5035,1
815400,199966,805,1


In [5]:
df_item = pd.read_csv('item_prices.zip').copy()
print(df_item.shape)
df_item.head()

(1000, 2)


,item_id,item_price
0,338,1501
1,74,647
2,7696,825
3,866,875
4,5876,804


In [6]:
df_item = df_item.drop_duplicates(subset=['item_id'], keep='last')
df_item.shape

(955, 2)

In [7]:
df_a.drop_duplicates()
df_c.drop_duplicates()
df_b.drop_duplicates()

print(df_a.shape)
print(df_c.shape)
print(df_b.shape)

(1188912, 3)
(1205510, 3)
(1198438, 3)


Все три выборки соизмеримы, утечки данных нет, так как id пользователей находятся в разных диапазонах, пересечений нет, дубли отсутсвуют. Создадим обработчики создав новые бинарные признаки на основе action_id:

In [8]:
def handler_0(action_id, zero):
    if action_id == 0:
        zero = action_id
        return zero
    else:
        return np.nan
    
    
def handler_1(action_id, one):
    if action_id == 1:
        one = action_id
        return one
    else:
        return np.nan
 
def handler_2(action_id, two):
    if action_id == 2:
        two = action_id
        return two   
    else:
        return np.nan

In [9]:
df_a['user_item'] = df_a.apply(lambda x: str(x.user_id) + '_' + str(x.item_id), axis=1)
df_a['one'], df_a['two'], df_a['zero'] = None, None, None
df_a['one'] = df_a.apply(lambda x: handler_1(x.action_id, x.one), axis=1)
df_a['two'] = df_a.apply(lambda x: handler_2(x.action_id, x.one), axis=1)
df_a['zero'] = df_a.apply(lambda x: handler_0(x.action_id, x.one), axis=1)

df_a = df_a.merge(
    df_item,
    how='left',
    on='item_id'    
)
df_a = df_a.groupby(['user_id', 'item_id', 'item_price'])[['one', 'two', 'zero']].count().reset_index()

df_c['user_item'] = df_c.apply(lambda x: str(x.user_id) + '_' + str(x.item_id), axis=1)
df_c['one'], df_c['two'], df_c['zero'] = None, None, None
df_c['one'] = df_c.apply(lambda x: handler_1(x.action_id, x.one), axis=1)
df_c['two'] = df_c.apply(lambda x: handler_2(x.action_id, x.one), axis=1)
df_c['zero'] = df_c.apply(lambda x: handler_0(x.action_id, x.one), axis=1)


df_c = df_c.merge(
    df_item,
    how='left',
    on='item_id'    
)
df_c = df_c.groupby(['user_id', 'item_id', 'item_price'])[['one', 'two', 'zero']].count().reset_index()

df_b['user_item'] = df_b.apply(lambda x: str(x.user_id) + '_' + str(x.item_id), axis=1)
df_b['one'], df_b['two'], df_b['zero'] = None, None, None
df_b['one'] = df_b.apply(lambda x: handler_1(x.action_id, x.one), axis=1)
df_b['two'] = df_b.apply(lambda x: handler_2(x.action_id, x.one), axis=1)
df_b['zero'] = df_b.apply(lambda x: handler_0(x.action_id, x.one), axis=1)


df_b = df_b.merge(
    df_item,
    how='left',
    on='item_id'    
)
df_b = df_b.groupby(['user_id', 'item_id', 'item_price'])[['one', 'two', 'zero']].count().reset_index()
print(df_a.shape)
print(df_c.shape)
print(df_b.shape)

(951130, 6)
(949221, 6)
(951141, 6)


Очистим некорректные данные:

- покупка без просмотра и клика
- клик без проcмотра и покупки
- просмотр и покупка без клика
- покупка и клик без просмотра 

In [10]:
# фильтруем  
df_a = df_a[((df_a['zero']==1) & (df_a['two']==1) & (df_a['one']==1)) |
     ((df_a['zero']!=1) & (df_a['two']!=1) & (df_a['one']==1)) |
     ((df_a['zero']==1) & (df_a['two']!=1) & (df_a['one']==1))]

# добавляем метрику gtm
df_a['gtm'] = round(df_a['two'] * df_a['item_price'])

# группируем по user_id
df_a = df_a.groupby(['user_id'])[['one', 'two', 'zero', 'gtm']].sum().reset_index()

# добавляем метрики ctr, purchase_rate 
df_a['ctr'] = round(df_a['zero'] / df_a['one'], 2)
df_a['purchase_rate'] = round(df_a['two'] / df_a['one'], 2)

df_a.head()

,user_id,one,two,zero,gtm,ctr,purchase_rate
0,241,918,10,179,8153,0.19,0.01
1,253,923,5,192,4848,0.21,0.01
2,362,913,8,178,8118,0.19,0.01
3,378,912,8,195,9197,0.21,0.01
4,475,912,7,197,6455,0.22,0.01


In [11]:
# фильтруем 
df_c = df_c[((df_c['zero']==1) & (df_c['two']==1) & (df_c['one']==1)) |
     ((df_c['zero']!=1) & (df_c['two']!=1) & (df_c['one']==1)) |
     ((df_c['zero']==1) & (df_c['two']!=1) & (df_c['one']==1))]

# добавляем метрику gtm
df_c['gtm'] = round(df_c['two'] * df_c['item_price'])

# группируем по user_id
df_c = df_c.groupby(['user_id'])[['one', 'two', 'zero', 'gtm']].sum().reset_index()

# добавляем метрики ctr, purchase_rate
df_c['ctr'] = round(df_c['zero'] / df_c['one'], 2)
df_c['purchase_rate'] = round(df_c['two'] / df_c['one'], 2)

df_c.head()

,user_id,one,two,zero,gtm,ctr,purchase_rate
0,200225,919,13,196,15503,0.21,0.01
1,200278,908,11,203,12427,0.22,0.01
2,200282,901,16,209,19945,0.23,0.02
3,200325,921,10,185,10261,0.20,0.01
4,200441,913,8,202,9003,0.22,0.01


In [12]:
# фильтруем  
df_b = df_b[((df_b['zero']==1) & (df_b['two']==1) & (df_b['one']==1)) |
     ((df_b['zero']!=1) & (df_b['two']!=1) & (df_b['one']==1)) |
     ((df_b['zero']==1) & (df_b['two']!=1) & (df_b['one']==1))]

# добавляем метрику gtm
df_b['gtm'] = round(df_b['two'] * df_b['item_price'])

# группируем по user_id
df_b = df_b.groupby(['user_id'])[['one', 'two', 'zero', 'gtm']].sum().reset_index()

# добавляем метрики ctr, purchase_rate
df_b['ctr'] = round(df_b['zero'] / df_b['one'], 2)
df_b['purchase_rate'] = round(df_b['two'] / df_b['one'], 2)

df_b.head()

,user_id,one,two,zero,gtm,ctr,purchase_rate
0,100038,874,19,167,19120,0.19,0.02
1,100099,859,23,161,22123,0.19,0.03
2,100164,865,8,140,7228,0.16,0.01
3,100321,877,18,180,18773,0.21,0.02
4,100397,891,17,153,15399,0.17,0.02


Проверим данные на нормальность, используем тест Шапиро — Уилка:

In [13]:
def shapiro_test(data, features):
    # задаём уровень значимости
    alpha = 0.01
    # загружаем данные
    # проводим тест Шапиро — Уилка
    for feature in features:
        _, p = stats.shapiro(data[feature])
        print('p-value = %.3f' % (p))
        # интерпретируем результат
        if p <= alpha:
            print('Распределение не нормальное')
        else:
            print('Распределение нормальное')

In [14]:
shapiro_test(df_a, ['gtm', 'ctr', 'purchase_rate'])
shapiro_test(df_c, ['gtm', 'ctr', 'purchase_rate'])
shapiro_test(df_c, ['gtm', 'ctr', 'purchase_rate'])

p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное
p-value = 0.000
Распределение не нормальное


По каждому интересующему признаку распределение не является нормальным, поэтому для дальнейшего анализа используем непараметрические тесты. Будем проводить два теста в кадом из которых будет две независимые группы поэтому используем U-КРИТЕРИЙ МАННА — УИТНИ:

In [15]:
def mannwhitneyu_test(data_1, data_2, features):

    # задаём уровень значимости
    alpha = 0.01
    # проводим тест
    for feature in features:
        print(f'Test for {feature}:')
        _, p = stats.mannwhitneyu(data_1[feature], data_2[feature])
        print('p-value = {:.3f}'.format(p))

        # интерпретируем результат
        if p <= alpha:
            print('p-значение меньше, чем заданный уровень значимости {:.2f}. Отвергаем нулевую гипотезу.'.format(alpha))
        else:
            print('p-значение больше, чем заданный уровень значимости {:.2f}. У нас нет оснований отвергнуть нулевую гипотезу.'.format(alpha))
            

Тест для групп А и С:

In [16]:
mannwhitneyu_test(df_a, df_c, ['gtm', 'ctr', 'purchase_rate'])

Test for gtm:
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Отвергаем нулевую гипотезу.
Test for ctr:
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Отвергаем нулевую гипотезу.
Test for purchase_rate:
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Отвергаем нулевую гипотезу.


In [17]:
fig = go.Figure()
fig.add_trace(go.Box(x=df_a['gtm'], name='А'))
fig.add_trace(go.Box(x=df_c['gtm'], name='С'))
fig.update_layout(title="Сравнение тестов А и С по gtm",
                  yaxis_title="Test",
                  xaxis_title="Gtm")
fig.show()

In [18]:
fig = go.Figure()
fig.add_trace(go.Box(x=df_a['ctr'], name='А'))
fig.add_trace(go.Box(x=df_c['ctr'], name='С'))
fig.update_layout(title="Сравнение тестов А и С по ctr",
                  yaxis_title="Test",
                  xaxis_title="ctr")
fig.show()

In [19]:
fig = go.Figure()
fig.add_trace(go.Box(x=df_a['purchase_rate'], name='А'))
fig.add_trace(go.Box(x=df_c['purchase_rate'], name='С'))
fig.update_layout(title="Сравнение тестов А и С по purchase_rate",
                  yaxis_title="Test",
                  xaxis_title="purchase_rate")
fig.show()

Сравнение групп А и В:

In [20]:
mannwhitneyu_test(df_a, df_b, ['gtm', 'ctr', 'purchase_rate'])

Test for gtm:
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Отвергаем нулевую гипотезу.
Test for ctr:
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Отвергаем нулевую гипотезу.
Test for purchase_rate:
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Отвергаем нулевую гипотезу.


In [21]:
fig = go.Figure()
fig.add_trace(go.Box(x=df_a['gtm'], name='А'))
fig.add_trace(go.Box(x=df_b['gtm'], name='B'))
fig.update_layout(title="Сравнение тестов А и B по gtm",
                  yaxis_title="Test",
                  xaxis_title="Gtm")
fig.show()

In [22]:
fig = go.Figure()
fig.add_trace(go.Box(x=df_a['ctr'], name='А'))
fig.add_trace(go.Box(x=df_b['ctr'], name='B'))
fig.update_layout(title="Сравнение тестов А и B по ctr",
                  yaxis_title="Test",
                  xaxis_title="ctr")
fig.show()

In [23]:
fig = go.Figure()
fig.add_trace(go.Box(x=df_a['purchase_rate'], name='А'))
fig.add_trace(go.Box(x=df_b['purchase_rate'], name='B'))
fig.update_layout(title="Сравнение тестов А и B по purchase_rate",
                  yaxis_title="Test",
                  xaxis_title="purchase_rate")
fig.show()

По результатам исследования A/A/B-тестирования можно сделать следующие выводы:

1. В А/А группах(df_a, df_c) показатели метрик ctr (отношение кликов к просмотрам товаров), purchase rate (отношение покупок к просмотрам товаров), gmv (оборот, сумма произведений количества покупок на стоимость покупки) имеют статистически значимые различия на уровне $alpha = 0.01$, что говорит о нестабилизации показателей в связи с недостаточно прошедшим временным периодом и/или некорректным разбиением пользователей на группы.

2. Метрики во всех группах имеют ненормальное распределение.

3. В свою очередь в А/В группах(df_a, df_b) тесты показали, что имеются статистически значимые различия в метриках, но вывод по данному тесту сделать неправильно так как не выполнились условия из п.1.
Графики boxplot подтверждают наши доводы.